In [64]:
#Import dependencies
import pandas as pd

import re
import requests as r

from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
#Data Input
synthetic_mgf = '/content/Synthetic_compounds_acyl_amides_renumbered (1).mgf'
library_mgf = '/content/library_massQL_acyl_amides_renumbered (1) (1) (1).mgf'

#Read in and index data to only include [M+H] peaks
def process_line(line):
    if line.startswith("NAME"):
        return [line.strip()]
    return []

def get_names(mgf_path):
    result = []
    with open(mgf_path, 'r') as file:
        for line in file:
            result = result + process_line(line)
    return result

synthetic_names = get_names(synthetic_mgf)
library_names = get_names(library_mgf)

indexed_synthetic_names = [[index + 1, value] for index, value in enumerate(synthetic_names) if value.endswith('[M+H]')]
indexed_library_names = [[index + 1, value] for index, value in enumerate(library_names) if value.endswith('[M+H]')]

#Create library and synthetic dataframes
library = pd.DataFrame(indexed_library_names, columns = ['index', 'base_string'])
synthetics = pd.DataFrame(indexed_synthetic_names, columns = ['index', 'base_string'])

In [ ]:
#Define functions to get N-acyl amide compound information
#(head group, carbon chain length, degree of unsaturation, number of acylations)
library_pattern = re.compile("-C\d+:\d")

def generate_properties_library(base_string):
    chain_match = library_pattern.search(base_string[15:])
    chain = chain_match.group(0)
    split_data = chain.split(":")
    head = base_string[15:15+chain_match.start(0)]
    return head, split_data[0][2:], split_data[1]

synthetic_pattern = re.compile("C\d+_\d+_N-Acyl_")
synthetic_pattern_diacyl = re.compile("C\d+:\d+ N,N-diacyl ")

def generate_properties_synthetic(base_string):
    chain_match = synthetic_pattern.search(base_string[5:])
    try:
        chain = chain_match.group(0)
        head = base_string[chain_match.end(0)+5:-6]
        chain_info = chain[1:-8].split("_")
        return head, chain_info[0], chain_info[1], 1
    except AttributeError:
        chain_match = synthetic_pattern_diacyl.search(base_string[5:])
        try:
            chain = chain_match.group(0)
        except:
            return "ERROR", "ERROR", "ERROR", 0
        head = base_string[chain_match.end(0)+5:-6]
        chain_info = chain[1:-12].split(":")
        return head, chain_info[0], chain_info[1], 2

In [ ]:
#Retrieve compound information
library['head'], library['chain_length'], library['unsaturation'] = zip(*library['base_string'].map(generate_properties_library))
synthetics['head'], synthetics['chain_length'], synthetics['unsaturation'], synthetics['num_acylations'] = zip(*synthetics['base_string'].map(generate_properties_synthetic))

library['head'] = library['head'].replace('2-phenethylamine','2-phenylethylamine')

library["compound"] = library["head"] + library["chain_length"].astype(str) + library["unsaturation"].astype(str)
synthetics["compound"] = synthetics["head"] + synthetics["chain_length"].astype(str) + synthetics["unsaturation"].astype(str)

#Filter synthetic standards to only include singularly acylated N-acyl amides
synthetics_filtered = synthetics[synthetics["num_acylations"]==1]

In [ ]:
#Retrieve indices of library compounds that match synthetic standards
def retrieve_indices(compound):
    indices = library.index[library['compound'] == compound].tolist()
    indices = [i+1 for i in indices]
    return indices

synthetics_filtered["library_matches"] = synthetics_filtered["compound"].apply(lambda x : retrieve_indices(x))
synthetics_filtered["temp_list"] = synthetics_filtered["index"].apply(lambda x : [x]) + synthetics_filtered["library_matches"]

In [ ]:
#Get mirror plot URLs
base_url = "https://metabolomics-usi.gnps2.org/svg/mirror/?usi1=mzspec%3AGNPS%3ATASK-9c69ce9fa9424d778160e4076ceaa063-spectra%2Fspecs_ms.mgf%3Ascan%3AREPLACE_SYNTHETIC&usi2=mzspec%3AGNPS%3ATASK-c3aca7e8842f457ca29a77f743b299fe-spectra%2Fspecs_ms.mgf%3Ascan%3AREPLACE_LIBRARY&width=10.0&height=6.0&mz_min=None&mz_max=None&max_intensity=125&annotate_precision=4&annotation_rotation=90&cosine=standard&fragment_mz_tolerance=0.1&grid=True&annotate_peaks=%5B%5B%5D%2C%20%5B%5D%5D"

def get_mirror_plots(indices):
    if len(indices) <= 1:
        return []
    usi_url = base_url.replace("REPLACE_SYNTHETIC",str(indices[0]))
    results = []
    library_indices = indices[1:]
    for i in library_indices:
        results.append(usi_url.replace("REPLACE_LIBRARY", str(i)))
    return results

synthetics_filtered["mirror_urls"] = synthetics_filtered["temp_list"].apply(lambda x : get_mirror_plots(x))

In [38]:
#Get cosine scores from mirror plot URLs
#URLs that fail to load in on first attempt have a value of -999999 as cosine score
def get_cosines(mirror_plot_links):
    results = []
    if len(mirror_plot_links) == 0:
        return results
    for link in mirror_plot_links:
        try:
            page = r.get(link)
            cosine = re.search('Cosine similarity = \d.\d+', page.text)
            results.append(float(cosine.group(0)[20:]))
        except:
            results.append(-999999)
    return results

synthetics_filtered["cosines"] = synthetics_filtered["mirror_urls"].progress_apply(lambda x : get_cosines(x))

In [60]:
#For each synthetic standard, retrieve the max cosine score and corresponding mirror plot
def max_plot(row):
    if len(row["cosines"]) > 0 and row["max_cosine"] != None:
        return row["mirror_urls"][row["cosines"].index(row["max_cosine"])]
    return None

synthetics_filtered["max_cosine"] = synthetics_filtered["cosines"].apply(lambda x : max(x) if len(x)!=0 else None)
synthetics_filtered["max_plot"] = synthetics_filtered.apply(lambda x : max_plot(x), axis=1)